In [1]:
import numpy as np
import pandas as pd
import re
import gc
from tqdm import tqdm
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
def count_encode(df, cols=[]):
    """
    count编码
    @param df:
    @param cols:
    @return:
    """
    for col in cols:
        print(col)
        vc = df[col].value_counts(dropna=True, normalize=True)
        df[col + '_count'] = df[col].map(vc).astype('float32')
    return df


def cross_cat_num(df, cat_col, num_col):
    """
    类别特征与数据特征groupby统计
    @param df:
    @param cat_col: 类别特征
    @param num_col: 数值特征
    @return:
    """
    def max_min(s):
        return s.max() - s.min()
    def quantile(s, q=0.25):
        return s.quantile(q)
    for f1 in tqdm(cat_col):
        g = df.groupby(f1, as_index=False)
        for f2 in tqdm(num_col):
            tmp = g[f2].agg({
                '{}_{}_count'.format(f1, f2): 'count',
                '{}_{}_max'.format(f1, f2): 'max',
                '{}_{}_min'.format(f1, f2): 'min',
                '{}_{}_median'.format(f1, f2): 'median',
                '{}_{}_mean'.format(f1, f2): 'mean',
                '{}_{}_sum'.format(f1, f2): 'sum',
                '{}_{}_skew'.format(f1, f2): 'skew',
                '{}_{}_std'.format(f1, f2): 'std',
                '{}_{}_nunique'.format(f1, f2): 'nunique',
                '{}_{}_max_min'.format(f1, f2): max_min,
                '{}_{}_quantile_25'.format(f1, f2): lambda x: quantile(x, 0.25),
                '{}_{}_quantile_75'.format(f1, f2): lambda x: quantile(x, 0.75)
            })
            df = df.merge(tmp, on=f1, how='left')
            del tmp
            gc.collect()
    return df


def arithmetic(df, cross_features):
    """
    数值特征之间的加减乘除
    @param df:
    @param cross_features: 交叉用的数值特征
    @return:
    """
    for i in tqdm(range(len(cross_features))):
        for j in range(i + 1, len(cross_features)):
            colname_add = '{}_{}_add'.format(cross_features[i], cross_features[j])
            colname_substract = '{}_{}_subtract'.format(cross_features[i], cross_features[j])
            colname_multiply = '{}_{}c_multiply'.format(cross_features[i], cross_features[j])
            df[colname_add] = df[cross_features[i]] + df[cross_features[j]]
            df[colname_substract] = df[cross_features[i]] - df[cross_features[j]]
            df[colname_multiply] = df[cross_features[i]] * df[cross_features[j]]

    for f1 in tqdm(cross_features):
        for f2 in cross_features:
            if f1 != f2:
                colname_ratio = '{}_{}_ratio'.format(f1, f2)
                df[colname_ratio] = df[f1].values / (df[f2].values + 0.001)
    return df

In [3]:
def get_psi(c, x_train, x_test):
    psi_res = pd.DataFrame()
    psi_dict={}
    # for c in tqdm(f_cols):
    try:
        t_train = x_train[c].fillna(-998)
        t_test = x_test[c].fillna(-998)
        #获取切分点
        bins=[]
        for i in np.arange(0,1.1,0.2):
            bins.append(t_train.quantile(i))
        bins=sorted(set(bins))
        bins[0]=-np.inf
        bins[-1]=np.inf
        #计算psi
        t_psi = pd.DataFrame()
        t_psi['train'] = pd.cut(t_train,bins).value_counts().sort_index()
        t_psi['test'] = pd.cut(t_test,bins).value_counts()
        t_psi.index=[str(x) for x in t_psi.index]
        t_psi.loc['总计',:] = t_psi.sum()
        t_psi['train_rate'] = t_psi['train']/t_psi.loc['总计','train']
        t_psi['test_rate'] = t_psi['test']/t_psi.loc['总计','test']
        t_psi['psi'] = (t_psi['test_rate']-t_psi['train_rate'])*(np.log(t_psi['test_rate'])-np.log(t_psi['train_rate']))
        t_psi.loc['总计','psi'] = t_psi['psi'].sum()
        t_psi.index.name=c
        #汇总
        t_res = pd.DataFrame([[c,t_psi.loc['总计','psi']]],
                             columns=['变量名','PSI'])
        psi_res = pd.concat([psi_res,t_res])
        psi_dict[c]=t_psi
        print(c,'done')
    except:
        print(c,'error')
    return psi_res #, psi_dict

In [4]:
def auc_select(X_train, y_train, X_valid, y_valid, cols, threshold=0.52):
    """
    基于AUC的单特征筛选
    @param X_train:
    @param y_train:
    @param X_valid:
    @param y_valid:
    @param cols:
    @return:
    """
    useful_cols = []
    useless_cols = []
    params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'metric': 'auc',
        # 'metric': 'None',  # 用自定义评估函数是将metric设置为'None'
        'learning_rate': 0.1,
        'num_leaves': 31,
        'lambda_l1': 0,
        'lambda_l2': 1,
        'num_threads': 23,
        'min_data_in_leaf': 20,
        'first_metric_only': True,
        'is_unbalance': True,
        'max_depth': -1,
        'seed': 2020
    }
    for i in cols:
        print(i)
        lgb_train = lgb.Dataset(X_train[[i]].values, y_train)
        lgb_valid = lgb.Dataset(X_valid[[i]].values, y_valid, reference=lgb_train)
        lgb_test = lgb.train(
            params,
            lgb_train,
            valid_sets=[lgb_valid, lgb_train],
            num_boost_round=1000,
            early_stopping_rounds=50,
            verbose_eval=20
        )
        print('*' * 10)
        print(lgb_test.best_score['valid_0']['auc'])
        if lgb_test.best_score['valid_0']['auc'] > threshold:
            useful_cols.append(i)
        else:
            useless_cols.append(i)
    return useful_cols, useless_cols

In [5]:
def correlation(df, threshold=0.98):
    """
    特征相关性计算
    @param df:
    @param threshold:
    @return:
    """
    col_corr = set()
    corr_matrix = df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colName = corr_matrix.columns[i]
                col_corr.add(colName)
    return col_corr

In [6]:
def train_func(train_path, test_path, save_path):
    # 请填写训练代码
    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)
    data = pd.concat([train, test])
    del train, test
    gc.collect()

    single_cols = ['appProtocol']
    data.drop(single_cols, axis=1, inplace=True)
    gc.collect()

    cat_cols = ['srcAddress', 'destAddress',
                'tlsVersion', 'tlsSubject', 'tlsIssuerDn', 'tlsSni']

    data['srcAddressPort'] = data['srcAddress'].astype(str) + data['srcPort'].astype(str)
    data['destAddressPort'] = data['destAddress'].astype(str) + data['destPort'].astype(str)
    
    # srcAddress To destAddress
    tmp = data.groupby('srcAddress', as_index=False)['destAddress'].agg({
        's2d_count': 'count',
        's2d_nunique': 'nunique'
    })
    data = data.merge(tmp, on='srcAddress', how='left')
    del tmp
    gc.collect()
    
    # srcAddressPort To destAddressPort
    tmp = data.groupby('srcAddressPort', as_index=False)['destAddressPort'].agg({
        'sp2dp_count': 'count',
        'sp2dp_nunique': 'nunique'
    })
    data = data.merge(tmp, on='srcAddressPort', how='left')
    del tmp
    gc.collect()
    
    # srcAddress To destAddressPort
    tmp = data.groupby('srcAddress', as_index=False)['destAddressPort'].agg({
        's2dp_count': 'count',
        's2dp_nunique': 'nunique'
    })
    data = data.merge(tmp, on='srcAddress', how='left')
    del tmp
    gc.collect()
    
    # srcAddressPort To destAddress
    tmp = data.groupby('srcAddressPort', as_index=False)['destAddress'].agg({
        'sp2d_count': 'count',
        'sp2d_nunique': 'nunique'
    })
    data = data.merge(tmp, on='srcAddressPort', how='left')
    del tmp
    gc.collect()
    
    # destAddress To srcAddress
    tmp = data.groupby('destAddress', as_index=False)['srcAddress'].agg({
        'd2s_count': 'count',
        'd2s_nunique': 'nunique'
    })
    data = data.merge(tmp, on='destAddress', how='left')
    del tmp
    gc.collect()
    
    # destAddressPort To srcAddressPort
    tmp = data.groupby('destAddressPort', as_index=False)['srcAddressPort'].agg({
        'dp2sp_count': 'count',
        'dp2sp_nunique': 'nunique'
    })
    data = data.merge(tmp, on='destAddressPort', how='left')
    del tmp
    gc.collect()
    
    # destAddressPort To srcAddress
    tmp = data.groupby('destAddressPort', as_index=False)['srcAddress'].agg({
        'dp2s_count': 'count',
        'dp2s_nunique': 'nunique'
    })
    data = data.merge(tmp, on='destAddressPort', how='left')
    del tmp
    gc.collect()
    
    # destAddress To srcAddressProt
    tmp = data.groupby('destAddress', as_index=False)['srcAddressPort'].agg({
        'd2sp_count': 'count',
        'd2sp_nunique': 'nunique'
    })
    data = data.merge(tmp, on='destAddress', how='left')
    del tmp
    gc.collect()
    
    cat_cols += ['srcAddressPort', 'destAddressPort']
    num_cols = ['bytesOut', 'bytesIn', 'pktsIn', 'pktsOut']
    data['bytesOut-bytesIn'] = data['bytesOut'] - data['bytesIn']
    data['pktsOut-pktsIn'] = data['pktsOut'] - data['pktsIn']
    
    
    tlsVersion_map = {
        'TLSv1': 1,
        'TLS 1.2': 1,
        'TLS 1.3': 1,
        'SSLv2': 2,
        'SSLv3': 3,
        '0x4854': 4,
        '0x4752': 4,
        'UNDETERMINED': 5
    }
    data['tlsVersion_map'] = data['tlsVersion'].map(tlsVersion_map)
    cat_cols.append('tlsVersion_map')

    data = count_encode(data, cat_cols)
    data = cross_cat_num(data, cat_cols, num_cols)
    data = arithmetic(data, num_cols)

    for i in cat_cols:
        lbl = LabelEncoder()
        data[i] = lbl.fit_transform(data[i].astype(str))
        data[i] = data[i].astype('category')

#     for i in ['srcPort', 'destPort']:
#         data[i] = data[i].astype('category')
    
    used_cols = [i for i in data.columns if i not in ['eventId', 'label']]
    train = data.loc[data['label'].notnull(), :]
    test = data.loc[data['label'].isnull(), :]
    sub = test[['eventId']]

    y = train['label']
    X = train[used_cols]
    X_test = test[used_cols]
    
    # 调用方法
    psi_res = Parallel(n_jobs=4)(delayed(get_psi)(c, X, X_test) for c in used_cols)
    psi_df = pd.concat(psi_res)
    used_cols = list(psi_df[psi_df['PSI'] <= 0.2]['变量名'].values)
    not_used_cols = list(psi_df[psi_df['PSI'] > 0.2]['变量名'].values)
    print('PSI drop features: \n', not_used_cols)
    
    X.drop(not_used_cols, axis=1, inplace=True)
    X_test.drop(not_used_cols, axis=1, inplace=True)
    
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=2020)
    
    print('y_train mean: ', y_train.mean())
    print('y_valid mean: ', y_valid.mean())
    
    useful_cols, useless_cols = auc_select(X_train, y_train, X_valid, y_valid, used_cols, threshold=0.52)
    print('AUC drop features: \n', useless_cols)
    
    X_train = X_train[useful_cols]
    X_valid = X_valid[useful_cols]
    X_test = X_test[useful_cols]
    
    col_corr = correlation(X_train, threshold=0.98)
    print('Correlation drop features: \n', col_corr)
    
    X_train.drop(col_corr, axis=1, inplace=True)
    X_valid.drop(col_corr, axis=1, inplace=True)
    X_test.drop(col_corr, axis=1, inplace=True)

    used_cols = X_train.columns.to_list()
    
    train_dataset = lgb.Dataset(X_train, y_train)
    valid_dataset = lgb.Dataset(X_valid, y_valid, reference=train_dataset)
    all_dataset = lgb.Dataset(train[used_cols], y, reference=train_dataset)

    params = {'objective': 'binary',
              'boosting': 'gbdt',
              'metric': 'auc',
              # 'metric': 'None',  # 用自定义评估函数是将metric设置为'None'
              'num_boost_round': 1000000,
              'learning_rate': 0.1,
              'num_leaves': 31,
              'lambda_l1': 0,
              'lambda_l2': 1,
              'num_threads': 23,
              'min_data_in_leaf': 20,
              'first_metric_only': True,
              'is_unbalance': True,
              'max_depth': -1,
              'seed': 2020}
    valid_model = lgb.train(params,
                            train_dataset,
                            valid_sets=[train_dataset, valid_dataset],
                            early_stopping_rounds=200,
                            verbose_eval=300)
    pred = valid_model.predict(X_valid)
    
    f1_best = 0
    for i in np.arange(0.1, 1, 0.01):
        y_valid_pred = np.where(pred > i, 1, 0)
        f1 = np.round(f1_score(y_valid, y_valid_pred), 5)
#         print('f1: ', f1)
        if f1 > f1_best:
            threshold = i
            f1_best = f1
        
    print('threshold: ', threshold)
    y_valid_pred = np.where(pred > threshold, 1, 0)
    print('Valid F1: ', np.round(f1_score(y_valid, y_valid_pred), 5))
    print('Valid mean label: ', np.mean(y_valid_pred))
#     binary_classification_report(y_valid, pred)
    
#     plt.figure(figsize=(8, 4))
#     ax = sns.kdeplot(pred, color='Red', shade=True)
#     ax.set_xlabel('pred')
#     ax.set_ylabel('Frequency')

    params = {'objective': 'binary',
              'boosting': 'gbdt',
              'metric': 'auc',
              # 'metric': 'None',  # 用自定义评估函数是将metric设置为'None'
              # 'num_boost_round': 1000000,
              'learning_rate': 0.1,
              'num_leaves': 31,
              'lambda_l1': 0,
              'lambda_l2': 1,
              'num_threads': 23,
              'min_data_in_leaf': 20,
              'first_metric_only': True,
              'is_unbalance': True,
              'max_depth': -1,
              'seed': 2020}
    train_model = lgb.train(params,
                            all_dataset,
                            num_boost_round=valid_model.best_iteration+20)
    y_test_pred = np.where(train_model.predict(X_test) > threshold, 1, 0)

    print('Test mean label: ', np.mean(y_test_pred))
    sub['label'] = y_test_pred
    sub.to_csv(save_path + '机器不学习原子弹也不学习_eta_submission_1020.csv', index=False)

In [7]:
if __name__ == '__main__':
    train_path = '../大数据队_eta_submission_1011/data/train.csv'
    test_path = '../大数据队_eta_submission_1011/data/test_1.csv'
    save_path = '../大数据队_eta_submission_1011/result/'
    train_func(train_path, test_path, save_path)

  0%|                                                                                            | 0/4 [00:00<?, ?it/s]

srcAddress
destAddress
tlsVersion
tlsSubject
tlsIssuerDn
tlsSni
srcAddressPort
destAddressPort
tlsVersion_map



100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 444.12it/s]


PSI drop features: 
 []
y_train mean:  0.09
y_valid mean:  0.09363636363636364
srcPort
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.936495	valid_0's auc: 0.932579
[40]	training's auc: 0.938502	valid_0's auc: 0.932585
Early stopping, best iteration is:
[4]	training's auc: 0.930248	valid_0's auc: 0.935111
Evaluated only: auc
**********
0.9351113534779094
destPort
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.642358	valid_0's auc: 0.618359
[40]	training's auc: 0.643573	valid_0's auc: 0.623748
[60]	training's auc: 0.643577	valid_0's auc: 0.623758
[80]	training's auc: 0.643578	valid_0's auc: 0.623757
[100]	training's auc: 0.643579	valid_0's auc: 0.623759
[120]	training's auc: 0.644376	valid_0's auc: 0.624224
[140]	training's auc: 0.644376	valid_0's auc: 0.624224
[160]	training's auc: 0.644377	valid_0's auc: 0.624224
Early stopping, best iteration is:
[118]	training's auc: 0.644376	valid_0's auc: 0.624224
Evaluated 

[40]	training's auc: 0.500995	valid_0's auc: 0.500903
Early stopping, best iteration is:
[1]	training's auc: 0.500995	valid_0's auc: 0.500903
Evaluated only: auc
**********
0.5009027081243731
d2sp_count
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.887986	valid_0's auc: 0.898681
[40]	training's auc: 0.888158	valid_0's auc: 0.898717
[60]	training's auc: 0.888185	valid_0's auc: 0.898991
[80]	training's auc: 0.888195	valid_0's auc: 0.899007
[100]	training's auc: 0.888195	valid_0's auc: 0.899007
[120]	training's auc: 0.88821	valid_0's auc: 0.899071
[140]	training's auc: 0.88821	valid_0's auc: 0.899071
[160]	training's auc: 0.888219	valid_0's auc: 0.899084
[180]	training's auc: 0.888241	valid_0's auc: 0.899118
[200]	training's auc: 0.888257	valid_0's auc: 0.899128
[220]	training's auc: 0.888268	valid_0's auc: 0.899166
[240]	training's auc: 0.888286	valid_0's auc: 0.899182
[260]	training's auc: 0.888307	valid_0's auc: 0.899192
[280]	training's auc: 0.888

...

[60]	training's auc: 0.868762	valid_0's auc: 0.872387
[80]	training's auc: 0.870823	valid_0's auc: 0.874077
[100]	training's auc: 0.871524	valid_0's auc: 0.874747
[120]	training's auc: 0.872464	valid_0's auc: 0.875077
[140]	training's auc: 0.872972	valid_0's auc: 0.875577
[160]	training's auc: 0.873238	valid_0's auc: 0.875908
[180]	training's auc: 0.874028	valid_0's auc: 0.87674
[200]	training's auc: 0.874227	valid_0's auc: 0.876952
[220]	training's auc: 0.874453	valid_0's auc: 0.877173
[240]	training's auc: 0.874786	valid_0's auc: 0.877439
[260]	training's auc: 0.874855	valid_0's auc: 0.877511
[280]	training's auc: 0.874924	valid_0's auc: 0.87761
[300]	training's auc: 0.875049	valid_0's auc: 0.877639
[320]	training's auc: 0.875367	valid_0's auc: 0.877911
[340]	training's auc: 0.875392	valid_0's auc: 0.877988
[360]	training's auc: 0.875624	valid_0's auc: 0.877319
Early stopping, best iteration is:
[317]	training's auc: 0.875363	valid_0's auc: 0.878034
Evaluated only: auc
**********
0.8

[40]	training's auc: 0.505352	valid_0's auc: 0.506018
Early stopping, best iteration is:
[1]	training's auc: 0.505352	valid_0's auc: 0.506018
Evaluated only: auc
**********
0.5060180541624875
tlsVersion_map_bytesIn_nunique
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.505352	valid_0's auc: 0.506018
[40]	training's auc: 0.505352	valid_0's auc: 0.506018
Early stopping, best iteration is:
[1]	training's auc: 0.505352	valid_0's auc: 0.506018
Evaluated only: auc
**********
0.5060180541624875
tlsVersion_map_bytesIn_max_min
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.505352	valid_0's auc: 0.506018
[40]	training's auc: 0.505352	valid_0's auc: 0.506018
Early stopping, best iteration is:
[1]	training's auc: 0.505352	valid_0's auc: 0.506018
Evaluated only: auc
**********
0.5060180541624875
tlsVersion_map_bytesIn_quantile_25
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.505319	valid_

Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.505352	valid_0's auc: 0.506018
[40]	training's auc: 0.505352	valid_0's auc: 0.506018
Early stopping, best iteration is:
[1]	training's auc: 0.505352	valid_0's auc: 0.506018
Evaluated only: auc
**********
0.5060180541624875
tlsVersion_map_pktsOut_max_min
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.505352	valid_0's auc: 0.506018
[40]	training's auc: 0.505352	valid_0's auc: 0.506018
Early stopping, best iteration is:
[1]	training's auc: 0.505352	valid_0's auc: 0.506018
Evaluated only: auc
**********
0.5060180541624875
tlsVersion_map_pktsOut_quantile_25
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.505318	valid_0's auc: 0.505918
[40]	training's auc: 0.505318	valid_0's auc: 0.505918
Early stopping, best iteration is:
[1]	training's auc: 0.505285	valid_0's auc: 0.505918
Evaluated only: auc
**********
0.5059177532597794
tlsVersion_ma

[40]	training's auc: 0.851654	valid_0's auc: 0.842005
[60]	training's auc: 0.853842	valid_0's auc: 0.8421
[80]	training's auc: 0.85515	valid_0's auc: 0.843127
[100]	training's auc: 0.856098	valid_0's auc: 0.842031
[120]	training's auc: 0.856386	valid_0's auc: 0.841284
Early stopping, best iteration is:
[81]	training's auc: 0.855165	valid_0's auc: 0.843144
Evaluated only: auc
**********
0.8431438003330379
pktsIn_pktsOut_add
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.841393	valid_0's auc: 0.832688
[40]	training's auc: 0.84464	valid_0's auc: 0.831726
Early stopping, best iteration is:
[4]	training's auc: 0.836648	valid_0's auc: 0.833917
Evaluated only: auc
**********
0.8339170910790624
pktsIn_pktsOut_subtract
Training until validation scores don't improve for 50 rounds
[20]	training's auc: 0.796077	valid_0's auc: 0.783978
[40]	training's auc: 0.79693	valid_0's auc: 0.7855
Early stopping, best iteration is:
[3]	training's auc: 0.792831	valid_0's auc

Correlation drop features: 
 {'bytesIn_pktsOut_subtract', 'srcAddress_pktsOut_quantile_25', 'destAddressPort_pktsOut_skew', 'srcAddressPort_pktsIn_quantile_75', 'd2sp_nunique', 'tlsVersion_pktsIn_median', 'tlsIssuerDn_pktsOut_sum', 'tlsSni_bytesOut_quantile_75', 'tlsSubject_pktsOut_max_min', 'destAddressPort_count', 'tlsIssuerDn_pktsOut_nunique', 'srcAddressPort_bytesIn_median', 'destAddressPort_pktsOut_nunique', 'tlsVersion_pktsIn_nunique', 'srcAddressPort_bytesOut_sum', 'srcAddressPort_pktsOut_quantile_75', 'tlsSni_pktsOut_count', 'tlsSni_bytesOut_max_min', 'tlsIssuerDn_pktsOut_max', 'tlsSni_bytesIn_count', 'tlsVersion_bytesIn_count', 'tlsVersion_pktsIn_max', 'destAddress_bytesIn_quantile_75', 'tlsVersion_pktsOut_skew', 'srcAddress_pktsOut_median', 'tlsIssuerDn_bytesIn_quantile_25', 'tlsIssuerDn_pktsOut_max_min', 'destAddressPort_bytesIn_median', 'destAddress_pktsIn_max_min', 'tlsVersion_pktsOut_nunique', 'srcAddressPort_bytesOut_count', 'destAddressPort_bytesOut_max', 'destAddressPo